# find unique counts
- aedavids@ucsc.edu
- 6/3/21

We can not use the salmon quant.sf files directly. The gencode.v35.ucsc.rmsk.salmon.v1.3.0.sid was created using 2 fasta files. The first contains the genecode reads. The names embed meta data about transcript id, gene id, and bio type. The second fasta file contains reads from the ucsc repeat masks. i.e. the TE's. The name is long string encoding position information.

to work around this problem see extraCellularRNA/R/notebooks/kras.ipsc.DESeq.normalize.v2.Rmd. It creates normalized count files. The counts estimated and are real number not integers. If a gene is not expressed it value will be zero

## quality control
Notice all the day seven sample have more genes. This is expected the cells are differentiating

In [1]:
import numpy as np
import pandas as pd
import pathlib as pl

In [2]:
# output from extraCellularRNA/R/notebooks/kras.ipsc.DESeq.normalize.v2.Rmd
dataRoot = '/private/home/aedavids/extraCellularRNA/data/R/output/normalizedCounts.gencode.v35.ucsc.rmsk.tx.to.gene.csv'

In [3]:
!ls -1 $dataRoot

kras.ipsc.day.5.DESeq.normalize.v2.gene.counts.csv
kras.ipsc.day.5.DESeq.normalize.v2.gene.sampleType_exo_vs_bulk.lfcShrink.csv
kras.ipsc.day.5.DESeq.normalize.v2.gene.sampleType_exo_vs_bulk.lfcShrink.meta.csv
kras.ipsc.day.5.DESeq.normalize.v2.gene.sampleType_exo_vs_bulk.unShrunk.csv
kras.ipsc.day.5.DESeq.normalize.v2.gene.sampleType_exo_vs_bulk.unShrunk.meta.csv
kras.ipsc.day.7.DESeq.normalize.v2.gene.counts.csv
kras.ipsc.day.7.DESeq.normalize.v2.gene.treatment_kras_vs_ctrl.lfcShrink.csv
kras.ipsc.day.7.DESeq.normalize.v2.gene.treatment_kras_vs_ctrl.lfcShrink.meta.csv
kras.ipsc.day.7.DESeq.normalize.v2.gene.treatment_kras_vs_ctrl.unShrunk.csv
kras.ipsc.day.7.DESeq.normalize.v2.gene.treatment_kras_vs_ctrl.unShrunk.meta.csv
pancreas.plasma.ev.long.RNA.normalized.deseq.biotype.counts.csv
pancreas.plasma.ev.long.RNA.normalized.deseq.gene.counts.csv
pancreas.plasma.ev.long.RNA.normalized.deseq.gene.counts.csv.bug
pancreas.plasma.ev.long.RNA.normalized.deseq.tx.counts.csv


## load and clean data

In [4]:
day5CountFile = pl.Path(dataRoot).joinpath('kras.ipsc.day.5.DESeq.normalize.v2.gene.counts.csv')
day7CountFile = pl.Path(dataRoot).joinpath('kras.ipsc.day.7.DESeq.normalize.v2.gene.counts.csv')
day5OutputFile = pl.Path("~/unmappedReadsAnalysis/data/numberOfUniqueGenesDay5SalmonMappedReads.tsv")
day7OutputFile = pl.Path("~/unmappedReadsAnalysis/data/numberOfUniqueGenesDay7SalmonMappedReads.tsv")

In [5]:
def countUniqueGenes( countFilePath ):
    '''
    countFilePath
        csv file
        
    returns (sampleNamesList, countsList)
    '''
    # load and clean data
    df  = pd.read_csv(countFilePath)
    df.rename(columns={'Unnamed: 0': 'gene'}, inplace=True)
    
    # count, skip the gene column
    sampleNamesList = df.columns[1:].to_list()
    colAxis = 0
    countsList =  np.count_nonzero(df.loc[:, sampleNamesList ], axis=colAxis)
    
    return(sampleNamesList, countsList)
    

In [6]:
sampleList5, countList5  = countUniqueGenes( day5CountFile )
print(sampleList5)
print(countList5)

['b_5_c_1', 'b_5_c_2', 'b_5_c_3', 'e_5_c_1', 'e_5_c_2', 'e_5_c_3', 'b_5_k_1', 'b_5_k_2', 'b_5_k_3', 'e_5_k_1', 'e_5_k_2', 'e_5_k_3']
[19813 19829 19825 13533 13797 13722 19820 19723 19752 16113 14336 14176]


In [7]:
sampleList7, countList7  = countUniqueGenes( day7CountFile )
print(sampleList7)
print(countList7)

['b_7_c_1', 'b_7_c_2', 'b_7_c_3', 'b_7_k_1', 'b_7_k_2', 'b_7_k_3']
[21152 21124 20933 20826 20854 21131]


In [8]:
def convertToDF( sampleNamesList, countsList ):
    '''
    makes it easy to write as csv file
    
    returns a dataframe
    '''
    dataDict = dict()
    for i in range(len(countsList)):
        dataDict[ sampleNamesList[i] ] = [countsList[i]]
    
    #print(dataDict)
    df =  pd.DataFrame( dataDict )
    return (df)

In [9]:
day5DF = convertToDF( sampleList5, countList5 )
day5DF

,b_5_c_1,b_5_c_2,b_5_c_3,e_5_c_1,e_5_c_2,e_5_c_3,b_5_k_1,b_5_k_2,b_5_k_3,e_5_k_1,e_5_k_2,e_5_k_3
0,19813,19829,19825,13533,13797,13722,19820,19723,19752,16113,14336,14176


In [10]:
day7DF = convertToDF( sampleList7, countList7 )
day7DF

,b_7_c_1,b_7_c_2,b_7_c_3,b_7_k_1,b_7_k_2,b_7_k_3
0,21152,21124,20933,20826,20854,21131


In [11]:
def saveCounts(df, inputFile, outputFile):
    tHeader = "./tmpHeader"
    tBody = "./tmpBody"
    !echo '#' $inputFile > $tHeader
    df.to_csv(tBody, sep="\t", index=False)
    !cat $tHeader $tBody > $outputFile
    !rm $tHeader $tBody


saveCounts(day5DF, day5CountFile, day5OutputFile)
saveCounts(day7DF, day7CountFile, day7OutputFile)

In [12]:
# test
!cat $day5OutputFile

# /private/home/aedavids/extraCellularRNA/data/R/output/normalizedCounts.gencode.v35.ucsc.rmsk.tx.to.gene.csv/kras.ipsc.day.5.DESeq.normalize.v2.gene.counts.csv
b_5_c_1	b_5_c_2	b_5_c_3	e_5_c_1	e_5_c_2	e_5_c_3	b_5_k_1	b_5_k_2	b_5_k_3	e_5_k_1	e_5_k_2	e_5_k_3
19813	19829	19825	13533	13797	13722	19820	19723	19752	16113	14336	14176


In [13]:
!cat $day7OutputFile

# /private/home/aedavids/extraCellularRNA/data/R/output/normalizedCounts.gencode.v35.ucsc.rmsk.tx.to.gene.csv/kras.ipsc.day.7.DESeq.normalize.v2.gene.counts.csv
b_7_c_1	b_7_c_2	b_7_c_3	b_7_k_1	b_7_k_2	b_7_k_3
21152	21124	20933	20826	20854	21131
